<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Modification of the span of an aircraft.

This notebook is based on the 3rd FAST-OAD-GA tutorial written by Aurélien. It aims to compare a reference architecture represented by the .XML file of an aircraft (Beechcraft 76, Cirrus 22, TBM-900) with a modified configuration. The modified architecture is produced by the FAST-OAD-GA module called modify_config which creates the corresponding .XML file. Then the .xml files of the reference aircraft and of the modified one go through all of the FAST-OAD-GA modules, with the generate_block_analysis function which calls each module. The variable viewer is called for both aircrafts after each module.

The only modification implemented for now is the increase of the wing span. The user has to define the multiplier of the span, and can also play on other parameters such as setting the engine position along the span.

The fastga code is from the root analysis_mode on GitHub.

Like in the tutorials, the structure of the notebook is based on data transfers between the data folder which stocks the .xml of the reference aircraft, and the workdir folder which stocks the .xml files passing through the modules of FAST-OAD-GA. After all the modules have been executed the final .xml files are placed in the output folder.

As the TBM-900 turbopropeller is still being modelized in FAST-OAD-GA, the Beechcraft 76 will be used for this analysis. There is no limit in the code for the increase in span, but some modules could have problems running with an extreme value (such as two times the original span).

## 1. Initialization of the work environment

In [1]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path.
current_path = os.getcwd()
DATA_FOLDER_PATH = pth.join(current_path, 'data')
WORK_FOLDER_PATH = pth.join(current_path, 'workdir')
OUTPUT_FOLDER_PATH = pth.join(current_path, 'output')

# # Clear work folder.
# shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)
# os.mkdir(WORK_FOLDER_PATH)

# For using more screen width.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Final file names. These .xml files will be overwritten each time they pass through a module.
AIRCRAFT_REF_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_reference.xml')
AIRCRAFT_MOD_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_mod.xml')

# Copy the reference aircraft file in the files of the work folder.
shutil.copy(pth.join(DATA_FOLDER_PATH, 'beechcraft_76.xml'), AIRCRAFT_REF_FILE)
shutil.copy(pth.join(DATA_FOLDER_PATH, 'beechcraft_76.xml'), AIRCRAFT_MOD_FILE)

Failed to import module fastga.models.aerodynamics.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.loops.unitary_tests.test_wing_area.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.geometry.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.performances.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.performances.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.mass_breakd

'C:\\Users\\Lucas\\FAST-OAD-GA\\FAST-GA\\src\\fastga\\notebooks\\Lucas\\workdir\\geometry_mod.xml'

## Propeller Computation

In [2]:
from fastga.models.aerodynamics.components.compute_propeller_aero import ComputePropellerPerformance

compute_propeller_ref = api_cs23.generate_block_analysis(
        ComputePropellerPerformance(),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

# compute_propeller_mod = api_cs23.generate_block_analysis(
#         ComputePropellerPerformance(),
#         [],
#         str(AIRCRAFT_MOD_FILE),
#         True,
#     )


output_ref = compute_propeller_ref({})
# output_mod = compute_propeller_mod({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

Entering propeller computation


2 Explicit Output(s) in 'system.propeller_aero_conditions'

varname                               value     
------------------------------------  ----------
data:aerodynamics:propeller:mach      [0.]      
data:aerodynamics:propeller:reynolds  [1000000.]


0 Implicit Output(s) in 'system.propeller_aero_conditions'




C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:478: RuntimeWarning: divide by zero encountered in true_divide
  (radius_max - radius) / radius * math.sqrt(1 + (omega * radius / (v_inf + v_i)) ** 2.0)
c:\users\lucas\fast-oad-ga\venvtest\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:358: UserWarning: Propeller not working in propulsive mode!
  warnings.warn("Propeller not working in propulsive mode!")
c:\users\lucas\fast-oad-ga\venvtest\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\Lucas\FAST

## 3. Geometry Module

In [3]:
from fastga.models.geometry import GeometryFixedTailDistance as Geometry

# Declare function
compute_geometry_ref = api_cs23.generate_block_analysis(
        Geometry(propulsion_id="fastga.wrapper.propulsion.basicIC_engine"),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

# Compute geometry
output_ref = compute_geometry_ref({})

In [ ]:
# Open viewer
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 4. Aerodynamics Module 

We choose to use the software OpenVSP, and we activate the mach interpolation option because it is needed to execute the load analysis module. We also compute the slipstream which allows to get more variables in cruise and to plot the lift coefficient along the span in the post-processing notebook.

This module usually represents the longest computational time.

In [4]:
from fastga.models.aerodynamics.aerodynamics import Aerodynamics

# Declare function on reference file
compute_aero_ref = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_cruise=True,
#             wing_airfoil_file="naca43013_3.af"
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )


# Compute both aircraft designs
output_ref = compute_aero_ref({})

# # Save Results
OUTPUT_AERO_FILE = pth.join(WORK_FOLDER_PATH, 'output_ref_aero.xml')
shutil.copy(AIRCRAFT_REF_FILE, OUTPUT_AERO_FILE)

'C:\\Users\\Lucas\\FAST-OAD-GA\\FAST-GA\\src\\fastga\\notebooks\\Lucas\\workdir\\output_ref_aero.xml'

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 5. MDA on the Weight and Performance Modules

In [ ]:
DATA_CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH,'mode_analyse.yml')
WORK_CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH,'mode_analyse.yml')
shutil.copy(DATA_CONFIGURATION_FILE, WORK_CONFIGURATION_FILE)

api_cs25.list_variables(WORK_CONFIGURATION_FILE)

In [ ]:
eval_problem = api_cs25.evaluate_problem(WORK_CONFIGURATION_FILE, overwrite=True)

OUTPUT_FILE = pth.join(WORK_FOLDER_PATH,'problem_outputs.xml')

In [ ]:
api_cs25.variable_viewer(OUTPUT_FILE)

In [ ]:
from fastga.models.weight.weight import Weight
from fastga.models.weight.mass_breakdown.update_mtow import UpdateMTOW
from fastga.models.performances.mission import Mission

# WEIGHT
compute_weight = api_cs23.generate_block_analysis(
        Weight(
            propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
        ),
        [],
        str(OUTPUT_FILE),
        True,
    )

output = compute_weight({})

# MTOW

compute_mtow = api_cs23.generate_block_analysis(
        UpdateMTOW(),
        [],
        str(OUTPUT_FILE),
        True,
    )

output = compute_mtow({})

# PERFORMANCES
compute_sizing_ref = api_cs23.generate_block_analysis(
        Mission(
            propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
        ),
        [],
        str(OUTPUT_FILE),
        True,
    )

output_ref = compute_sizing_ref({})

In [ ]:
api_cs25.variable_viewer(OUTPUT_FILE)

## 6. Load Analysis Module

In [ ]:
from fastga.models.load_analysis.loads import Loads

compute_loads_ref = api_cs23.generate_block_analysis(
        Loads(
            propulsion_id="fastga.wrapper.propulsion.basic_turboprop",
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

# compute_loads_mod = api_cs23.generate_block_analysis(
#         Loads(
#             propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
#         ),
#         [],
#         str(AIRCRAFT_MOD_FILE),
#         True,
#     )

output_ref = compute_loads_ref({})
# output_mod = compute_loads_mod({})

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 7. Handling Qualities Module

As the main purpose of this notebook is to directly visualize the variations of the plane performance and its handling qualities by modifying the reference architecture, the static margin will be computed without updating the tail areas.

In [ ]:
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities

compute_hq_ref = api_cs23.generate_block_analysis(
        ComputeHandlingQualities(
            propulsion_id="fastga.wrapper.propulsion.basic_turboprop",
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

# compute_hq_mod = api_cs23.generate_block_analysis(
#         ComputeHandlingQualities(
#             propulsion_id="fastga.wrapper.propulsion.basicIC_engine",
#         ),
#         [],
#         str(AIRCRAFT_MOD_FILE),
#         True,
#     )

output_ref = compute_hq_ref({})
# output_mod = compute_hq_mod({})

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## Save files for post processing

In [ ]:
# Clear output folder.
shutil.rmtree(OUTPUT_FOLDER_PATH, ignore_errors=True)
os.mkdir(OUTPUT_FOLDER_PATH)

OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod.xml')
shutil.copy(AIRCRAFT_REF_FILE, OUTPUT_REF_FILE)
shutil.copy(AIRCRAFT_MOD_FILE, OUTPUT_MOD_FILE)